In [16]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [17]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [18]:
from utils import load_data, convert_landmarks_to_eye_movements

all_data, all_labels = load_data('miami_deception')

In [19]:
converted_data, blinks = convert_landmarks_to_eye_movements(all_data)

## Ograniczenie do pierwszych N klatek i normalizacja per próbkę

In [ ]:
import numpy as np

def normalize_sample(data):
    mean = np.mean(data, axis=(0, 1), keepdims=True)
    std = np.std(data, axis=(0, 1), keepdims=True)
    normalized = (data - mean) / (std + 1e-8)
    return normalized

In [21]:
normalized_cut_data = np.array([normalize_sample(d[:1000, :]) for d in converted_data], dtype=object)

## Preprocessing danych

In [22]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(normalized_cut_data, all_labels, binarize_labels=False)

In [23]:
print(X_train.shape)
print(y_train.shape)

torch.Size([224, 1000, 2])
torch.Size([224])


In [24]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 160
1: 160
=============><=============


In [36]:
X_train[4].min()

tensor(-1.9428)

# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [25]:
import torch
import torch.nn as nn

In [26]:
class EyeDirectionTransformerClassifier(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        
        self.input_proj = nn.Linear(2, d_model)  # Project (dir_x, dir_y) to d_model
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.cls_head = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Linear(32, 1)  # Binary classification
        )

    def forward(self, x):
        # x: [batch, seq_len, 2]
        x = self.input_proj(x)  # -> [batch, seq_len, d_model]
        x = self.transformer_encoder(x)  # -> [batch, seq_len, d_model]
        x = x.mean(dim=1)  # Global average pooling over time
        out = self.cls_head(x)  # -> [batch, 1]
        return out


In [27]:
from torch.optim import Adam

model = EyeDirectionTransformerClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_weight = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [28]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs',  model.__class__.__qualname__, 'eye_movement')
writer = SummaryWriter(writer_path)

In [29]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [30]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=128, unbalanced=True, show_prediction_stats=False, epochs=1000)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                     TRAINING                                         VALIDATION                    
----------------------------------------------------------------------------------------------------
Loss        : 1.3687                                    Loss        : 0.7066
Accuracy    : 0.4509                                    Accuracy    : 0.3542
Precision   : 0.2254                                    Precision   : 0.1771
Recall      : 0.5000                                    Recall      : 0.5000
F1 Score    : 0.3108                                    F1 Score    : 0.2615
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                   

## Ewaluacja modelu

In [31]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 3.0837
Accuracy    : 0.5000
Precision   : 0.4934
Recall      : 0.4939
F1 Score    : 0.4857
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 0.6400                                    FP Rate     : 0.6522



# TODYNET

### Przygotowanie danych

In [15]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [16]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "MIAMI_DECEPTION_EYE_MOVEMENT")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [17]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_np, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [18]:
X_train_tensor.shape

torch.Size([224, 1, 1679, 2])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='MIAMI_DECEPTION_EYE_MOVEMENT' --num_layers 1 --in_dim 16 --hidden_dim 16 --out_dim 16 --pool_ratio 0.0 --kern_size "3" --groups 1